## import dependencies and data file

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

#import warnings
#warnings.filterwarnings('ignore')

In [2]:
OSMI_MHIT_cleaned = pd.read_csv(Path('../resources/OSMI_MHIT_mostly_clean.csv'))
OSMI_MHIT_cleaned

,new_id,self_employed,company_size,tech_company,mh_coverage,mh_coverage_awareness,mh_employer_discussion,mh_resources_provided,mh_anonimity,mh_medical_leave,...,age,gender,country_live,live_us_state,country_work,work_us_state,work_position,remote,quantile_age_1,quantile_age_2
0,1,0,26-100,1,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,...,39,male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes,"(38.0, 99.0]","(37.0, 39.0]"
1,2,0,6-25,1,No,Yes,Yes,Yes,Yes,Somewhat easy,...,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never,"(28.0, 32.0]","(27.0, 29.0]"
2,3,0,6-25,1,No,NaN,No,No,I don't know,Neither easy nor difficult,...,38,male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always,"(32.0, 38.0]","(37.0, 39.0]"
3,4,0,6-25,0,Yes,Yes,No,No,No,Neither easy nor difficult,...,43,female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes,"(38.0, 99.0]","(39.0, 44.0]"
4,5,0,More than 1000,1,Yes,I am not sure,No,Yes,Yes,Somewhat easy,...,42,male,United Kingdom,NaN,United Kingdom,NaN,DevOps/SysAdmin|Support|Back-end Developer|Fro...,Sometimes,"(38.0, 99.0]","(39.0, 44.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,1003,0,100-500,1,I don't know,I am not sure,No,I don't know,I don't know,Somewhat easy,...,26,female,Canada,NaN,Canada,NaN,Other,Sometimes,"(19.999, 28.0]","(25.0, 27.0]"
1000,1004,0,500-1000,1,Yes,No,No,No,Yes,Somewhat easy,...,38,female,United States of America,Illinois,United States of America,Illinois,Support,Always,"(32.0, 38.0]","(37.0, 39.0]"
1001,1005,0,100-500,1,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,...,52,male,United States of America,Georgia,United States of America,Georgia,Back-end Developer,Sometimes,"(38.0, 99.0]","(44.0, 99.0]"
1002,1006,0,100-500,0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,...,30,female,United States of America,Nebraska,United States of America,Nebraska,DevOps/SysAdmin,Sometimes,"(28.0, 32.0]","(29.0, 30.0]"


In [3]:
OSMI_MHIT_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004 entries, 0 to 1003
Data columns (total 54 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   new_id                                 1004 non-null   int64 
 1   self_employed                          1004 non-null   int64 
 2   company_size                           1004 non-null   object
 3   tech_company                           1004 non-null   int64 
 4   mh_coverage                            1004 non-null   object
 5   mh_coverage_awareness                  901 non-null    object
 6   mh_employer_discussion                 1004 non-null   object
 7   mh_resources_provided                  1004 non-null   object
 8   mh_anonimity                           1004 non-null   object
 9   mh_medical_leave                       1004 non-null   object
 10  mh_discussion_negative_impact          1004 non-null   object
 11  ph_discussion_neg

In [6]:
OSMI_MHIT_cleaned['mh_coverage_awareness'].value_counts()

I am not sure    309
Yes              297
No               295
Name: mh_coverage_awareness, dtype: int64